In [37]:
from google import google
import gnp
import codecs
import json
from boto.s3.connection import S3Connection
from boto.s3.key import Key
import boto
import sys, os
import pandas as pd
from TwitterSearch import *
import sunlight
from sunlight import congress
from bs4 import BeautifulSoup
import urllib2
import itertools
import datetime
import configparser
import time
from pattern.en import parsetree
from pattern.en import mood

from googleplaces import GooglePlaces, types, lang
import csv
import pandas as pd
import time
import sys
import requests
import urllib, simplejson
from pprint import pprint

In [138]:
cities = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'Top5000Population.csv'))
cities['full_city'] = '\"' + cities['city'] + ', ' + cities['state'] + '\"'
print cities[:10]
city_list = list(cities['full_city'])
print city_list[:10]
print city_list.index('Jersey City, NJ')

            city state population            full_city
0      New York     NY  8,363,710      "New York , NY"
1   Los Angeles     CA  3,833,995   "Los Angeles , CA"
2       Chicago     IL  2,853,114       "Chicago , IL"
3       Houston     TX  2,242,193       "Houston , TX"
4       Phoenix     AZ  1,567,924       "Phoenix , AZ"
5  Philadelphia     PA  1,447,395  "Philadelphia , PA"
6   San Antonio     TX  1,351,305   "San Antonio , TX"
7        Dallas     TX  1,279,910        "Dallas , TX"
8     San Diego     CA  1,279,329     "San Diego , CA"
9      San Jose     CA    948,279      "San Jose , CA"
['"New York , NY"', '"Los Angeles , CA"', '"Chicago , IL"', '"Houston , TX"', '"Phoenix , AZ"', '"Philadelphia , PA"', '"San Antonio , TX"', '"Dallas , TX"', '"San Diego , CA"', '"San Jose , CA"']


ValueError: 'Jersey City, NJ' is not in list

In [114]:

action_categories = {
                     'Townhall': ["indivisible",'\"town hall\"'],
#                        'Donation': ["donation","donate","give support","financial support","contribute","contributing","give a contribution"],
                     'Protest': ["protest","rally","demonstration"],
#                      'Gathering': ["event"],
#                                    ,"meetup","huddle","congregate","gather","gathering","discuss","discussion", "indivisible"],
                     'Boycott': ["boycott"]
#                      'Advocate': ["call","email","reach out", "sign petition","petition"],
#                      'Vote': ["vote","cast your ballot"]
                          }

issue_types = { 'immigrant rights' : ['immigrant rights',"immigrants' rights",'refugee rights', 'travel ban',
                                        'border wall','refugees','asylum','immigration reform',
                                        'immigrant advocacy','migrant rights','undocumented'],
               "women's rights" : ["women's rights","women's rights",'womens rights','gender equality'
                                  'girl power',"international women's day","war on women","planned parenthood"],
               "black rights" : ["racial equality","black lives matter","african american rights","civil rights",
                                "black power","jim crow"],
               "LGTBQ rights" : ["marriage equality","transgender rights","equality act","lesbian rights",'gay rights',
                                'bisexual rights'],
               "voting rights": ["redistricting","gerrymandering","redistrict","gerrymander","voter id","voting access",
                                "voter access","voter suppression"]
                      }


legislators = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'legislators.csv'))

mask = (legislators.in_office == 1)

legislators = legislators.loc[mask]

legislators['title_name'] = '\"' + legislators['full_title'] + ' ' + legislators['firstname'] + ' ' + legislators['lastname'] + '\"'

# print legislators.columns
# print
# print list(legislators['title_name'])[:10]

leg_names = list(legislators['title_name'])
# leg_states = list(legislators['state'])

keyword_searches = []
for category in action_categories:
    for combination in itertools.product( action_categories[category],city_list[5:]):
      keyword_searches.append(list(combination))
  
print keyword_searches[:20]

[['protest', '"Nashville, TN"'], ['protest', '"Washington , DC"'], ['protest', '"Las Vegas , NV"'], ['protest', '"Portland , OR"'], ['protest', '"Louisville, KY"'], ['protest', '"Oklahoma City , OK"'], ['protest', '"Tucson , AZ"'], ['protest', '"Atlanta , GA"'], ['protest', '"Albuquerque , NM"'], ['protest', '"Fresno , CA"'], ['protest', '"Sacramento , CA"'], ['protest', '"Long Beach , CA"'], ['protest', '"Mesa , AZ"'], ['protest', '"Kansas City , MO"'], ['protest', '"Omaha , NE"'], ['protest', '"Cleveland , OH"'], ['protest', '"Virginia Beach , VA"'], ['protest', '"Miami , FL"'], ['protest', '"Oakland , CA"'], ['protest', '"Raleigh , NC"']]


In [44]:
url = 'https://www.facebook.com/Dayton-Womens-Rights-Alliance-589846831145259/'
header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
req = urllib2.Request(url,headers=header)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

/usr/local/lib/python2.7/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [73]:
divs = soup.findAll('div',attrs={'class':'userContent'})
for div in divs:
  ps = div.findAll('p')
  for p in ps:
    print p.text
    print
#     if p.find('a'):
#       print p.find('a')['href']
#       print
#       if p.find('a',attrs={'class':'_5inf'}):
#         print p.find('a',attrs={'class':'_5inf'})['href']
#         print p.find('a',attrs={'class':'_5inf'})['title']
#       print
#       print

  

Here are 3 ways you can participate in #DayWithoutAWoman on March 8th: 1. Women take the day off, from paid and unpaid labor 2. Avoid shopping for one day (with exceptions for small, women- and minority-owned businesses) 3. Wear RED in solidarity https://www.bustle.com/p/the-day-without-a-woman-strike-det…

https://www.flippable.org/blog/2017/2/…/the-unseen-elections

Get the real story on health care and the ACA

If we don't have a planet, the rest of this doesn't matter!! Friends: the House GOP has filed a bill to eliminate the Environmental Protection Agency, effective December 2018. The EPA performs critical functions to protect human and environmental public health. I know it's tempting to be focused on Flynn, intelligence leaks, and Cabinet appointees at this time, but we should remain vigilant about substantive things on the GOP agenda. If you want to protect the EPA, here are 3... things you can do: 1. If you live in Florida's first Congressional district, call your Congressman

In [146]:
# sources = ['facebook','change.org']
' AND '.join(keyword_searches[0]) + ' AND \"facebook event\"'

'protest AND "Nashville, TN" AND "facebook event"'

In [154]:

num_page = 2
nresults = 0
today = datetime.datetime.today()
weeks_ago = today - datetime.timedelta(days=14)

with open(os.path.join(os.pardir, 'data/documents', 'google_real_data_' + today.strftime("%Y%m%d%H%M") + '.txt'), 'w') as outfile:
  outfile.write('[')
  for category in action_categories.keys():
    for city in city_list[77:]:
      try: 
        search_results = google.search(city + ' AND (' + ' OR '.join(action_categories[category]) + ') AND \"facebook event\"', num_page)
        for result in search_results:
           ### Extract Date from Facebook Event Description if it exists
          if result.link.startswith('https://www.facebook.com/events/'):
            facebook_event_id = result.link.split("events/",1)[1][:-1]  
            try:
              event_date = dparser.parse(result.description,fuzzy=True).strftime("%m/%d/%Y")
            except:
              event_date = None

            try:
              json_doc = {
                                'posted_date': datetime.datetime.today().strftime("%m/%d/%Y"),
                                'expiration_date': event_date,
                                'action_keywords': category, 
                                'issues_keywords': None, 
                                'document': {'content': result.description,
                                           'title': result.name,
                                           'summary': None,
                                           'url': result.link,
                                           'date': event_date,
                                        },
                                'location': {'addr1': None,
                                           'addr2': None,
                                           'city':  city.split(',',1)[0],
                                           'state': city.split(',',1)[1],
                                           'zip':  None ,
                                           'lat':   None,
                                           'long':  None,
                                           },
                                'org_name': None,

                                # your mileage may vary
                                'other_metadata': {
                                    'google_cached_page': result.cached,
                                    'google_search_link': result.google_link
                                    }
                            }

              new_record = {
                                    "_index" : "facebook",
                                    "_type"  : "event",
                                    "_id"    : facebook_event_id,
                                    "_source": json_doc,
                                }

              json.dump(new_record, outfile)
              outfile.write(',')
              ntweets += 1
              if ntweets % 100:
                print(result.description)
            except:
              print sys.exc_info()[0]

          ### Extract Date from Facebook Event Description if it exists
          elif result.link.startswith('https://www.facebook.com/'):
            facebook_organization = result.description.split("|",1)[0][:-1]
            facebook_id = result.link.split("facebook.com/",1)[1][:-1]  


            try:
              json_doc = {
                                'posted_date': datetime.datetime.today().strftime("%m/%d/%Y"),
                                'expiration_date': None,
                                'action_keywords': category, 
                                'issues_keywords': None, 
                                'document': {'content': result.description,
                                           'title': result.name,
                                           'summary': None,
                                           'url': result.link,
                                           'date': None
                                        },
                                'location': {'addr1': None,
                                           'addr2': None,
                                           'city':  city.split(',',1)[0],
                                           'state': city.split(',',1)[1],
                                           'zip':  None ,
                                           'lat':   None,
                                           'long':  None,
                                           },
                                'org_name': facebook_organization,

                                # your mileage may vary
                                'other_metadata': {
                                    'google_cached_page': result.cached,
                                    'google_search_link': result.google_link
                                    }
                            }

              new_record = {
                                    "_index" : "facebook",
                                    "_type"  : "group",
                                    "_id"    : facebook_id,
                                    "_source": json_doc,
                                }

              json.dump(new_record, outfile)
              outfile.write(',')
              ntweets += 1
              if ntweets % 100:
                print(result.description)
            except:
              print sys.exc_info()[0]

          else:
            try:
              event_date = dparser.parse(result.description,fuzzy=True).strftime("%m/%d/%Y")
            except:
              event_date = None  

            try:
              json_doc = {
                                'posted_date': datetime.datetime.today().strftime("%m/%d/%Y"),
                                'expiration_date': event_date,
                                'action_keywords': category, 
                                'issues_keywords': None, 
                                'document': {'content': result.description,
                                           'title': result.name,
                                           'summary': None,
                                           'url': result.link,
                                           'date': event_date
                                        },
                                'location': {'addr1': None,
                                           'addr2': None,
                                           'city':  city.split(',',1)[0],
                                           'state': city.split(',',1)[1],
                                           'zip':  None ,
                                           'lat':   None,
                                           'long':  None,
                                           },
                                'org_name': None,

                                # your mileage may vary
                                'other_metadata': {
                                    'google_cached_page': result.cached,
                                    'google_search_link': result.google_link
                                    }
                            }

              new_record = {
                                    "_index" : "facebook",
                                    "_type"  : "group",
                                    "_id"    : facebook_id,
                                    "_source": json_doc,
                                }

              json.dump(new_record, outfile)
              outfile.write(',')
              ntweets += 1
              if ntweets % 100:
                print(result.description)
            except:
              print sys.exc_info()[0]
#               time.sleep(3*60)
      except:
        print sys.exc_info()[0]
        break

  outfile.seek(-1, os.SEEK_END)
  outfile.truncate()
  outfile.write(']')




<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exceptions.NameError'>
<type 'exc

In [3]:
config = configparser.ConfigParser()
config.read(os.path.join(os.pardir, 'config', 'ross.ini'))

consumer_key = config['twitter.api']['consumer_key']
consumer_secret = config['twitter.api']['consumer_secret']
access_token = config['twitter.api']['access_token']
access_token_secret = config['twitter.api']['access_token_secret']

AWS_ACCESS_KEY_ID = config['aws.creds']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = config['aws.creds']['AWS_SECRET_ACCESS_KEY']

YOUR_API_KEY = config['google.api']['api_key']

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  from ipykernel import kernelapp as app


In [4]:
reload(sys)  
sys.setdefaultencoding('UTF8')


zip_geo = pd.read_csv('~/w210_imwithdata/data/static_data/Zip_lat_long.csv')

zips = []

for index, row in zip_geo.iterrows():
    new_record = {
        'zip': row['ZIP'],
        'lat': row['LAT'],
        'lng': row['LNG']
    }
    zips.append(new_record)

In [5]:
zips[:10]

[{'lat': 18.180554999999998, 'lng': -66.749960999999999, 'zip': 601.0},
 {'lat': 18.361945000000002, 'lng': -67.175596999999996, 'zip': 602.0},
 {'lat': 18.455182999999998, 'lng': -67.119887000000006, 'zip': 603.0},
 {'lat': 18.158345000000001, 'lng': -66.932911000000004, 'zip': 606.0},
 {'lat': 18.295366000000001, 'lng': -67.125135, 'zip': 610.0},
 {'lat': 18.402252999999998, 'lng': -66.711396999999991, 'zip': 612.0},
 {'lat': 18.420411999999999, 'lng': -66.671979000000007, 'zip': 616.0},
 {'lat': 18.445146999999999, 'lng': -66.559696000000002, 'zip': 617.0},
 {'lat': 17.991245000000003, 'lng': -67.153993, 'zip': 622.0},
 {'lat': 18.083361, 'lng': -67.153897000000001, 'zip': 623.0}]

In [ ]:
def GoogPlac(lat,lng,keyword,radius,language,pagetoken,key):
  #making the url
    if pagetoken is str:
        AUTH_KEY = key
        LOCATION = str(lat)+ "," + str(lng)
        KEYWORD =keyword
        RADIUS = radius
        LANGUAGE = language
        PAGETOKEN = pagetoken
        MyUrl = ('https://maps.googleapis.com/maps/api/place/nearbysearch/json'
                '?location=%s'
                    '&radius=%s'
                    '&keyword=%s'
                '&language=%s'
                '&page_token=%s'
                '&sensor=false&key=%s') % (LOCATION, RADIUS, KEYWORD, LANGUAGE,PAGETOKEN, AUTH_KEY)
        #grabbing the JSON result
    else:
        AUTH_KEY = key
        LOCATION = str(lat)+ "," + str(lng)
        KEYWORD =keyword
        RADIUS = radius
        LANGUAGE = language
        MyUrl = ('https://maps.googleapis.com/maps/api/place/nearbysearch/json'
                '?location=%s'
                    '&radius=%s'
                    '&keyword=%s'
                '&language=%s'
                '&sensor=false&key=%s') % (LOCATION, RADIUS, KEYWORD, LANGUAGE, AUTH_KEY)
        #grabbing the JSON result
        response = urllib.urlopen(MyUrl)
        jsonRaw = response.read()
        jsonData = json.loads(jsonRaw)
        return jsonData


def GoogDetails(place_id,key):
  #making the url
  AUTH_KEY = key
  PLACEID = place_id
  MyUrl = ('https://maps.googleapis.com/maps/api/place/details/json'
           '?placeid=%s'
           '&sensor=false&key=%s') % (PLACEID, AUTH_KEY)
  #grabbing the JSON result
  response = urllib.urlopen(MyUrl)
  jsonRaw = response.read()
  jsonData = json.loads(jsonRaw)
  return jsonData

google_place_ids = []

i = 0
with open('data_google_places1.txt', 'a') as outfile:
  for search in keyword_searches:
      for i in xrange(0,len(zips),30):
  #         if zips[i]['zip'] in range(60600,60702):
              #if i >= 434:
              # Retry Query for a Max of 3 pages, depending on whether the query has a next_page_token
            for k in range(3):
                # First page will not include a next_page_token in the query
                if k == 0:
                    # Grab the JSON result
                    content = GoogPlac(zips[i]['lat'],zips[i]['lng'],' '.join(search),'50000','en',False,YOUR_API_KEY)
                    # Check for a next page token
                    next_page_token = content.get('next_page_token',None)
                    # Loop through the places and get the details necessary to populate the final records for Placement
                    for place in content['results']:
                        # Query Google Details with the Place ID
                        details = GoogDetails(place['place_id'],YOUR_API_KEY)
                        new_record = {
                                            'source': 'Google',
                                            'keywords': search,
                                            'name': details['result'].get('name','None'),
                                            'google_place_id': place['place_id'],
                                            'address': details['result'].get('formatted_address','None'),
                                            'phone': details['result'].get('formatted_phone_number','None'),
                                            'latitude': place['geometry']['location']['lat'],
                                            'longitude': place['geometry']['location']['lng'],
                                            'google_url': details['result'].get('url','None'),
                                            'website':details['result'].get('website','None'),
                                            }
                        if place['place_id'] not in google_place_ids:
                        # Append the record to the output list
                          json.dump(new_record, outfile)
                          outfile.write(',')
                          outfile.write('\n')
                          google_place_ids.append(place['place_id'])
                        # Keep track of the output
                        if len(google_place_ids) % 100 == 0:
                            print len(google_place_ids)
                            print new_record
                # If the original query has a next page token, then query the next page
                if k > 0 and next_page_token: 
                    # Add the next page token to the query
                    content = GoogPlac(zips[i]['lat'],zips[i]['lng'],' '.join(search),'50000','en',next_page_token,YOUR_API_KEY)
                    # If there's content, then get the new records
                    if content:
                        next_page_token = content.get('next_page_token',None)
                        # Checking that this part of the code was actually working, so  print page # if afte first page
                        # Google only lets you get to 60 results per query, or 3 pages, so it will never be > 3
                        print 'in page %s of results' % (k+1)
                        for place in content['results']:
                            # Query Google Details with the Place ID
                            details = GoogDetails(place['place_id'],YOUR_API_KEY)
                            new_record = {
                                                'source': 'Google',
                                                'keywords': search,
                                                ''
                                                'name': details['result'].get('name','None'),
                                                'google_place_id': place['place_id'],
                                                'address': details['result'].get('formatted_address','None'),
                                                'phone': details['result'].get('formatted_phone_number','None'),
                                                'latitude': place['geometry']['location']['lat'],
                                                'longitude': place['geometry']['location']['lng'],
                                                'google_url': details['result'].get('url','None'),
                                                'website':details['result'].get('website','None'),
                                                }
                            # Append the record to the output list
                            if place['place_id'] not in google_place_ids:
                            # Append the record to the output list
                              json.dump(new_record, outfile)
                              outfile.write(',')
                              outfile.write('\n')
                              google_place_ids.append(place['place_id'])
                            # Keep track of the output
                            if len(google_place_ids) % 100 == 0:
                                print len(google_place_ids)
                                print new_record
                    else:
                        break


In [123]:
# bucket_name = 'mids-capstone-rzst'
# conn = boto.connect_s3(AWS_ACCESS_KEY_ID,AWS_SECRET_ACCESS_KEY)

# def percent_cb(complete, total):
#     sys.stdout.write('.')
#     sys.stdout.flush()

# bucket = conn.get_bucket('mids-capstone-rzst')
# key = boto.s3.key.Key(bucket, os.path.join(os.pardir,"data_google_places1.txt"))
# key.set_contents_from_filename(os.path.join(os.pardir,"data_google_places1.txt"))
# key.set_acl('public-read')

In [124]:
os.pardir

'..'

In [121]:
# bucket_name = 'mids-capstone-rzst'
# conn = boto.connect_s3(AWS_ACCESS_KEY_ID,AWS_SECRET_ACCESS_KEY)


# # bucket = conn.create_bucket(bucket_name,
# #     location=boto.s3.connection.Location.DEFAULT)

# testfile = os.path.join(os.pardir,"data4.txt")
# print 'Uploading %s to Amazon S3 bucket %s' % \
#    (testfile, bucket_name)

# def percent_cb(complete, total):
#     sys.stdout.write('.')
#     sys.stdout.flush()


# # k = Key(bucket_name)
# # k.key = 'data.txt'
# # k.set_contents_from_filename(testfile,
# #     cb=percent_cb, num_cb=10)


# # s3_connection = boto.connect_s3()
# bucket = conn.get_bucket('mids-capstone-rzst')
# key = boto.s3.key.Key(bucket, os.path.join(os.pardir,"data4.txt"))
# key.set_contents_from_filename(os.path.join(os.pardir,"data4.txt"))
# key.set_acl('public-read')

In [122]:
# filein = os.path.join(os.pardir,"data_google_places1.txt") 

# f = open(filein,'r')
# filedata = f.read()
# f.close()

# newdata = filedata.replace("\n",",")

# fileout = os.path.join(os.pardir,"data_google_places2.json") 

# f = open(fileout,'w+')
# f.write('[')
# f.write(newdata)
# f.seek(-1, os.SEEK_END)
# f.truncate()
# f.write(']')
# f.close()


# d = simplejson.loads(open(os.path.join(os.pardir,"data_google_places2.json")).read())

    

In [80]:
d[0]

{'address': 'Alden St, Springfield, MA 01109, USA',
 'google_place_id': 'ChIJZ_alNrLn5okRbrIugLIZnQo',
 'google_url': 'https://maps.google.com/?cid=764795766168269422',
 'keywords': ['donation', 'civil rights'],
 'latitude': 42.10410779999999,
 'longitude': -72.5534139,
 'name': 'NAACP Springfield',
 'phone': 'None',
 'source': 'Google',
 'website': 'None'}